### Initial data read-in/prep

In [ ]:
# Update sklearn 

# !pip install sklearn --upgrade

In [ ]:
# Update tensorflow

# !pip install tensorflow --upgrade

In [1]:
# Import dependencies

from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder, MinMaxScaler,StandardScaler
from sklearn import tree
from sklearn.datasets import load_iris
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
from numpy import asarray
import os
%matplotlib inline
import matplotlib.pyplot as plt

C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# Read the census csv into a pandas DataFrame

census = pd.read_csv('../Resources/Tableau_clean/census_all.csv')
census.head()

,State,year,Name,Population,HHI,PovertyRate
0,1,2011,Alabama,4747424,42934,17.133186
1,2,2011,Alaska,700703,69014,9.292239
2,4,2011,Arizona,6337373,50752,15.835820
3,5,2011,Arkansas,2895928,40149,17.846507
4,6,2011,California,36969200,61632,14.096818


In [3]:
# rename State column for joining and drop the "State" column

census = census.rename(columns={'Name': 'state_name'})
del census["State"]
census

,year,state_name,Population,HHI,PovertyRate
0,2011,Alabama,4747424,42934,17.133186
1,2011,Alaska,700703,69014,9.292239
2,2011,Arizona,6337373,50752,15.835820
3,2011,Arkansas,2895928,40149,17.846507
4,2011,California,36969200,61632,14.096818
...,...,...,...,...,...
250,2015,Virginia,8256630,65015,11.164628
251,2015,Washington,6985464,61062,13.005750
252,2015,West Virginia,1851420,41751,17.466809
253,2015,Wisconsin,5742117,53357,12.614651


In [4]:
# Read the medicare csv into a pandas DataFrame

medicare = pd.read_csv('../Resources/Tableau_clean/medicare_all.csv')
del medicare['state_id']
medicare

,year,state_name,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,average_diabetic_enrollees_blood_lipids_test,average_female_enrollees_age_67_to_69,average_female_age_67_to_69_mammogram,beneficiaries_part_a_eligible,leg_amputations_per_1000_enrollees,discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees
0,2011,Alabama,478784.0,82.5,69691.0,83.9,63.3,80.3,42267.0,62.8,501422.0,0.99,76.4
1,2011,Alaska,49648.0,70.9,5449.0,74.5,55.8,66.9,5151.0,55.3,54928.0,0.81,53.1
2,2011,Arizona,465298.0,78.2,50991.0,78.5,66.1,75.8,43614.0,64.3,501103.0,0.57,51.4
3,2011,Arkansas,327939.0,80.7,40202.0,82.5,66.1,76.2,29290.0,58.3,345431.0,0.85,77.0
4,2011,California,2238140.0,72.7,243999.0,80.6,64.1,78.0,190971.0,59.0,2378472.0,0.63,49.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2015,Virginia,832699.0,83.3,98165.0,87.6,70.0,82.0,83135.0,64.5,703266.0,0.53,42.8
251,2015,Washington,610922.0,78.5,56474.0,86.6,67.7,76.3,56238.0,60.0,510796.0,0.46,32.7
252,2015,West Virginia,206961.0,80.8,29239.0,84.7,59.9,78.2,20777.0,59.1,171837.0,0.90,75.0
253,2015,Wisconsin,474364.0,79.8,46596.0,91.0,69.7,81.6,44595.0,71.9,418646.0,0.65,45.0


In [5]:
pollution = pd.read_csv('../Resources/Tableau_clean/pollution_all.csv')
pollution

,state_name,Days_with_AQI,Year,Good_Days,Moderate_Days,Unhealthy_Days,Very_Unhealthy_Days,Hazardous_Days,Days_CO,Days_NO2,Days_Ozone,Days_SO2,Days_PM2.5,Days_PM10
0,Alabama,248.53,2011,171.79,72.37,0.37,0.00,0.37,0.11,0.00,158.63,0.16,84.89,4.74
1,Alaska,274.50,2011,235.25,34.88,0.62,0.12,0.62,2.12,0.00,75.62,0.00,176.25,20.50
2,Arizona,320.62,2011,177.15,117.00,2.38,1.00,2.38,0.00,3.15,167.15,16.54,27.77,106.00
3,Arkansas,216.88,2011,154.47,59.18,0.12,0.00,0.12,0.00,3.00,108.76,14.76,90.35,0.00
4,California,345.65,2011,193.17,120.37,7.02,0.37,7.02,0.17,6.91,197.43,0.09,129.24,11.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,Virginia,261.00,2015,228.14,32.34,0.00,0.00,0.00,1.14,35.06,141.26,0.97,73.66,8.91
251,Washington,345.83,2015,292.76,48.14,1.03,0.07,1.03,0.00,2.69,47.45,0.00,290.21,5.48
252,West Virginia,280.47,2015,234.07,45.20,0.07,0.00,0.07,0.13,0.00,145.53,40.47,81.20,13.13
253,Wisconsin,291.79,2015,245.86,42.68,0.07,0.00,0.07,0.00,0.71,191.43,14.54,84.86,0.25


In [6]:
# Identify column names for joining

list(medicare.columns) 

['year',
 'state_name',
 'beneficiaries_part_b',
 'one_ambulatory_visit',
 'diabetic_enrollees_age_65_to_75',
 'average_diabetic_enrollees_hemoglobin_a1c_test',
 'average_diabetic_enrollees_eye_exam',
 'average_diabetic_enrollees_blood_lipids_test',
 'average_female_enrollees_age_67_to_69',
 'average_female_age_67_to_69_mammogram',
 'beneficiaries_part_a_eligible',
 'leg_amputations_per_1000_enrollees',
 'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees']

In [7]:
# join census and medicare csvs

data = census.join(medicare, on=None, how='left', lsuffix='state_name', rsuffix='state_name', sort=False)
data

,yearstate_name,state_namestate_name,Population,HHI,PovertyRate,yearstate_name,state_namestate_name,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,average_diabetic_enrollees_blood_lipids_test,average_female_enrollees_age_67_to_69,average_female_age_67_to_69_mammogram,beneficiaries_part_a_eligible,leg_amputations_per_1000_enrollees,discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees
0,2011,Alabama,4747424,42934,17.133186,2011,Alabama,478784.0,82.5,69691.0,83.9,63.3,80.3,42267.0,62.8,501422.0,0.99,76.4
1,2011,Alaska,700703,69014,9.292239,2011,Alaska,49648.0,70.9,5449.0,74.5,55.8,66.9,5151.0,55.3,54928.0,0.81,53.1
2,2011,Arizona,6337373,50752,15.835820,2011,Arizona,465298.0,78.2,50991.0,78.5,66.1,75.8,43614.0,64.3,501103.0,0.57,51.4
3,2011,Arkansas,2895928,40149,17.846507,2011,Arkansas,327939.0,80.7,40202.0,82.5,66.1,76.2,29290.0,58.3,345431.0,0.85,77.0
4,2011,California,36969200,61632,14.096818,2011,California,2238140.0,72.7,243999.0,80.6,64.1,78.0,190971.0,59.0,2378472.0,0.63,49.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2015,Virginia,8256630,65015,11.164628,2015,Virginia,832699.0,83.3,98165.0,87.6,70.0,82.0,83135.0,64.5,703266.0,0.53,42.8
251,2015,Washington,6985464,61062,13.005750,2015,Washington,610922.0,78.5,56474.0,86.6,67.7,76.3,56238.0,60.0,510796.0,0.46,32.7
252,2015,West Virginia,1851420,41751,17.466809,2015,West Virginia,206961.0,80.8,29239.0,84.7,59.9,78.2,20777.0,59.1,171837.0,0.90,75.0
253,2015,Wisconsin,5742117,53357,12.614651,2015,Wisconsin,474364.0,79.8,46596.0,91.0,69.7,81.6,44595.0,71.9,418646.0,0.65,45.0


In [8]:
# drop dupe columns

dropped = data.loc[:,~data.columns.duplicated()]
dropped

,yearstate_name,state_namestate_name,Population,HHI,PovertyRate,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,average_diabetic_enrollees_blood_lipids_test,average_female_enrollees_age_67_to_69,average_female_age_67_to_69_mammogram,beneficiaries_part_a_eligible,leg_amputations_per_1000_enrollees,discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees
0,2011,Alabama,4747424,42934,17.133186,478784.0,82.5,69691.0,83.9,63.3,80.3,42267.0,62.8,501422.0,0.99,76.4
1,2011,Alaska,700703,69014,9.292239,49648.0,70.9,5449.0,74.5,55.8,66.9,5151.0,55.3,54928.0,0.81,53.1
2,2011,Arizona,6337373,50752,15.835820,465298.0,78.2,50991.0,78.5,66.1,75.8,43614.0,64.3,501103.0,0.57,51.4
3,2011,Arkansas,2895928,40149,17.846507,327939.0,80.7,40202.0,82.5,66.1,76.2,29290.0,58.3,345431.0,0.85,77.0
4,2011,California,36969200,61632,14.096818,2238140.0,72.7,243999.0,80.6,64.1,78.0,190971.0,59.0,2378472.0,0.63,49.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2015,Virginia,8256630,65015,11.164628,832699.0,83.3,98165.0,87.6,70.0,82.0,83135.0,64.5,703266.0,0.53,42.8
251,2015,Washington,6985464,61062,13.005750,610922.0,78.5,56474.0,86.6,67.7,76.3,56238.0,60.0,510796.0,0.46,32.7
252,2015,West Virginia,1851420,41751,17.466809,206961.0,80.8,29239.0,84.7,59.9,78.2,20777.0,59.1,171837.0,0.90,75.0
253,2015,Wisconsin,5742117,53357,12.614651,474364.0,79.8,46596.0,91.0,69.7,81.6,44595.0,71.9,418646.0,0.65,45.0


In [9]:
# rename columns

dropped = dropped.rename(columns={'state_namestate_name': 'state_name', 'yearstate_name':'Year'})
dropped

,Year,state_name,Population,HHI,PovertyRate,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,average_diabetic_enrollees_blood_lipids_test,average_female_enrollees_age_67_to_69,average_female_age_67_to_69_mammogram,beneficiaries_part_a_eligible,leg_amputations_per_1000_enrollees,discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees
0,2011,Alabama,4747424,42934,17.133186,478784.0,82.5,69691.0,83.9,63.3,80.3,42267.0,62.8,501422.0,0.99,76.4
1,2011,Alaska,700703,69014,9.292239,49648.0,70.9,5449.0,74.5,55.8,66.9,5151.0,55.3,54928.0,0.81,53.1
2,2011,Arizona,6337373,50752,15.835820,465298.0,78.2,50991.0,78.5,66.1,75.8,43614.0,64.3,501103.0,0.57,51.4
3,2011,Arkansas,2895928,40149,17.846507,327939.0,80.7,40202.0,82.5,66.1,76.2,29290.0,58.3,345431.0,0.85,77.0
4,2011,California,36969200,61632,14.096818,2238140.0,72.7,243999.0,80.6,64.1,78.0,190971.0,59.0,2378472.0,0.63,49.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2015,Virginia,8256630,65015,11.164628,832699.0,83.3,98165.0,87.6,70.0,82.0,83135.0,64.5,703266.0,0.53,42.8
251,2015,Washington,6985464,61062,13.005750,610922.0,78.5,56474.0,86.6,67.7,76.3,56238.0,60.0,510796.0,0.46,32.7
252,2015,West Virginia,1851420,41751,17.466809,206961.0,80.8,29239.0,84.7,59.9,78.2,20777.0,59.1,171837.0,0.90,75.0
253,2015,Wisconsin,5742117,53357,12.614651,474364.0,79.8,46596.0,91.0,69.7,81.6,44595.0,71.9,418646.0,0.65,45.0


In [10]:
# join pollution csv
data_2 = dropped.join(pollution, on=None, how='inner', lsuffix='state_name', rsuffix='state_name', sort=False)
data_2

,Yearstate_name,state_namestate_name,Population,HHI,PovertyRate,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,...,Moderate_Days,Unhealthy_Days,Very_Unhealthy_Days,Hazardous_Days,Days_CO,Days_NO2,Days_Ozone,Days_SO2,Days_PM2.5,Days_PM10
0,2011,Alabama,4747424,42934,17.133186,478784.0,82.5,69691.0,83.9,63.3,...,72.37,0.37,0.00,0.37,0.11,0.00,158.63,0.16,84.89,4.74
1,2011,Alaska,700703,69014,9.292239,49648.0,70.9,5449.0,74.5,55.8,...,34.88,0.62,0.12,0.62,2.12,0.00,75.62,0.00,176.25,20.50
2,2011,Arizona,6337373,50752,15.835820,465298.0,78.2,50991.0,78.5,66.1,...,117.00,2.38,1.00,2.38,0.00,3.15,167.15,16.54,27.77,106.00
3,2011,Arkansas,2895928,40149,17.846507,327939.0,80.7,40202.0,82.5,66.1,...,59.18,0.12,0.00,0.12,0.00,3.00,108.76,14.76,90.35,0.00
4,2011,California,36969200,61632,14.096818,2238140.0,72.7,243999.0,80.6,64.1,...,120.37,7.02,0.37,7.02,0.17,6.91,197.43,0.09,129.24,11.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2015,Virginia,8256630,65015,11.164628,832699.0,83.3,98165.0,87.6,70.0,...,32.34,0.00,0.00,0.00,1.14,35.06,141.26,0.97,73.66,8.91
251,2015,Washington,6985464,61062,13.005750,610922.0,78.5,56474.0,86.6,67.7,...,48.14,1.03,0.07,1.03,0.00,2.69,47.45,0.00,290.21,5.48
252,2015,West Virginia,1851420,41751,17.466809,206961.0,80.8,29239.0,84.7,59.9,...,45.20,0.07,0.00,0.07,0.13,0.00,145.53,40.47,81.20,13.13
253,2015,Wisconsin,5742117,53357,12.614651,474364.0,79.8,46596.0,91.0,69.7,...,42.68,0.07,0.00,0.07,0.00,0.71,191.43,14.54,84.86,0.25


In [11]:
# rename columns

data_all = data_2.rename(columns={'state_namestate_name': 'State', 'Yearstate_name':'Year'})
data_all

,Year,State,Population,HHI,PovertyRate,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,...,Moderate_Days,Unhealthy_Days,Very_Unhealthy_Days,Hazardous_Days,Days_CO,Days_NO2,Days_Ozone,Days_SO2,Days_PM2.5,Days_PM10
0,2011,Alabama,4747424,42934,17.133186,478784.0,82.5,69691.0,83.9,63.3,...,72.37,0.37,0.00,0.37,0.11,0.00,158.63,0.16,84.89,4.74
1,2011,Alaska,700703,69014,9.292239,49648.0,70.9,5449.0,74.5,55.8,...,34.88,0.62,0.12,0.62,2.12,0.00,75.62,0.00,176.25,20.50
2,2011,Arizona,6337373,50752,15.835820,465298.0,78.2,50991.0,78.5,66.1,...,117.00,2.38,1.00,2.38,0.00,3.15,167.15,16.54,27.77,106.00
3,2011,Arkansas,2895928,40149,17.846507,327939.0,80.7,40202.0,82.5,66.1,...,59.18,0.12,0.00,0.12,0.00,3.00,108.76,14.76,90.35,0.00
4,2011,California,36969200,61632,14.096818,2238140.0,72.7,243999.0,80.6,64.1,...,120.37,7.02,0.37,7.02,0.17,6.91,197.43,0.09,129.24,11.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2015,Virginia,8256630,65015,11.164628,832699.0,83.3,98165.0,87.6,70.0,...,32.34,0.00,0.00,0.00,1.14,35.06,141.26,0.97,73.66,8.91
251,2015,Washington,6985464,61062,13.005750,610922.0,78.5,56474.0,86.6,67.7,...,48.14,1.03,0.07,1.03,0.00,2.69,47.45,0.00,290.21,5.48
252,2015,West Virginia,1851420,41751,17.466809,206961.0,80.8,29239.0,84.7,59.9,...,45.20,0.07,0.00,0.07,0.13,0.00,145.53,40.47,81.20,13.13
253,2015,Wisconsin,5742117,53357,12.614651,474364.0,79.8,46596.0,91.0,69.7,...,42.68,0.07,0.00,0.07,0.00,0.71,191.43,14.54,84.86,0.25


### Trial 1: Comparing Poverty and Population to Medicare Part B Beneficiaries

### Set features

In [12]:
# Identify column names 

list(data_all.columns) 

['Year',
 'State',
 'Population',
 'HHI',
 'PovertyRate',
 'beneficiaries_part_b',
 'one_ambulatory_visit',
 'diabetic_enrollees_age_65_to_75',
 'average_diabetic_enrollees_hemoglobin_a1c_test',
 'average_diabetic_enrollees_eye_exam',
 'average_diabetic_enrollees_blood_lipids_test',
 'average_female_enrollees_age_67_to_69',
 'average_female_age_67_to_69_mammogram',
 'beneficiaries_part_a_eligible',
 'leg_amputations_per_1000_enrollees',
 'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees',
 'State',
 'Days_with_AQI',
 'Year',
 'Good_Days',
 'Moderate_Days',
 'Unhealthy_Days',
 'Very_Unhealthy_Days',
 'Hazardous_Days',
 'Days_CO',
 'Days_NO2',
 'Days_Ozone',
 'Days_SO2',
 'Days_PM2.5',
 'Days_PM10']

In [13]:
features = data_all[['Population', 'PovertyRate']]

In [30]:
y_data = data_all.iloc[:,5]

In [31]:
# get X and y variables

y = y_data
# output of model
X = features

### Train/Test/Split

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,Population,PovertyRate
224,5930538,9.726015
144,6402387,17.208144
108,3583561,9.888153
178,6028076,15.134033
73,9897264,15.984438


### StandardScaler

In [42]:
# scale the data
X_scaler =  StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# # Label-encode data set - removed due to error "y contains previously unseen labels:"
# label_encoder = LabelEncoder()
# label_encoder.fit(y_train)
# encoded_y_train = label_encoder.transform(y_train)
# encoded_y_test = label_encoder.transform(y_test)


# print(encoded_y_test)

### Training the model

In [43]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, y_train)
predictions = model.predict(X_test_scaled)

In [44]:
print(f"Training Data Score: {model.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.0
Testing Data Score: 0.0


In [46]:
# Calculate classification report - determines whether it is a good fit for your model

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

# low precision =:(
# can play with different features and y-values for better scores

              precision    recall  f1-score   support

     58583.0       0.00      0.00      0.00       0.0
     61807.0       0.00      0.00      0.00       1.0
     64431.0       0.00      0.00      0.00       1.0
     68066.0       0.00      0.00      0.00       1.0
     72959.0       0.00      0.00      0.00       0.0
     76856.0       0.00      0.00      0.00       1.0
     77205.0       0.00      0.00      0.00       1.0
     77318.0       0.00      0.00      0.00       0.0
     77669.0       0.00      0.00      0.00       0.0
     80820.0       0.00      0.00      0.00       1.0
     82210.0       0.00      0.00      0.00       0.0
     83038.0       0.00      0.00      0.00       1.0
     85947.0       0.00      0.00      0.00       1.0
     88192.0       0.00      0.00      0.00       1.0
     88616.0       0.00      0.00      0.00       1.0
     91053.0       0.00      0.00      0.00       0.0
     91929.0       0.00      0.00      0.00       0.0
     98609.0       0.00    

C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Feature Importance

In [55]:
beneficiaries = y_data

In [60]:
target = y_data
beneficiaries_list = y_data.tolist()
target_names = [beneficiaries_list]

In [61]:
data = data_all.drop("beneficiaries_part_b", axis=1)
data = data.drop("State", axis=1)
data

,Year,Population,HHI,PovertyRate,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,average_diabetic_enrollees_blood_lipids_test,average_female_enrollees_age_67_to_69,...,Moderate_Days,Unhealthy_Days,Very_Unhealthy_Days,Hazardous_Days,Days_CO,Days_NO2,Days_Ozone,Days_SO2,Days_PM2.5,Days_PM10
0,2011,4747424,42934,17.133186,82.5,69691.0,83.9,63.3,80.3,42267.0,...,72.37,0.37,0.00,0.37,0.11,0.00,158.63,0.16,84.89,4.74
1,2011,700703,69014,9.292239,70.9,5449.0,74.5,55.8,66.9,5151.0,...,34.88,0.62,0.12,0.62,2.12,0.00,75.62,0.00,176.25,20.50
2,2011,6337373,50752,15.835820,78.2,50991.0,78.5,66.1,75.8,43614.0,...,117.00,2.38,1.00,2.38,0.00,3.15,167.15,16.54,27.77,106.00
3,2011,2895928,40149,17.846507,80.7,40202.0,82.5,66.1,76.2,29290.0,...,59.18,0.12,0.00,0.12,0.00,3.00,108.76,14.76,90.35,0.00
4,2011,36969200,61632,14.096818,72.7,243999.0,80.6,64.1,78.0,190971.0,...,120.37,7.02,0.37,7.02,0.17,6.91,197.43,0.09,129.24,11.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2015,8256630,65015,11.164628,83.3,98165.0,87.6,70.0,82.0,83135.0,...,32.34,0.00,0.00,0.00,1.14,35.06,141.26,0.97,73.66,8.91
251,2015,6985464,61062,13.005750,78.5,56474.0,86.6,67.7,76.3,56238.0,...,48.14,1.03,0.07,1.03,0.00,2.69,47.45,0.00,290.21,5.48
252,2015,1851420,41751,17.466809,80.8,29239.0,84.7,59.9,78.2,20777.0,...,45.20,0.07,0.00,0.07,0.13,0.00,145.53,40.47,81.20,13.13
253,2015,5742117,53357,12.614651,79.8,46596.0,91.0,69.7,81.6,44595.0,...,42.68,0.07,0.00,0.07,0.00,0.71,191.43,14.54,84.86,0.25


In [62]:
feature_names = data.columns

In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [64]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.0

In [65]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)
# aiming at over .9

0.0

In [66]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.0480335935724723, 'Days_PM2.5'),
 (0.047352226924083876, 'HHI'),
 (0.04495383313623589, 'PovertyRate'),
 (0.04358903368966272, 'one_ambulatory_visit'),
 (0.04352201802741451, 'average_diabetic_enrollees_hemoglobin_a1c_test'),
 (0.04330278707556242, 'Days_Ozone'),
 (0.04239753443441562, 'average_diabetic_enrollees_blood_lipids_test'),
 (0.04239177252775733, 'average_diabetic_enrollees_eye_exam'),
 (0.042302445008156046, 'Population'),
 (0.041869567171107756,
  'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees'),
 (0.0409865086566159, 'average_female_age_67_to_69_mammogram'),
 (0.04013519153397497, 'Good_Days'),
 (0.03911452976314388, 'leg_amputations_per_1000_enrollees'),
 (0.03893713777080266, 'Moderate_Days'),
 (0.038730849925447786, 'diabetic_enrollees_age_65_to_75'),
 (0.03860781356143842, 'Days_NO2'),
 (0.038234961592612825, 'Days_with_AQI'),
 (0.03786308623803976, 'average_female_enrollees_age_67_to_69'),
 (0.03682134284704798, 'beneficiaries_part_a_eligible'

### Trial 2: Comparing Ambulatory Discharges and Ambulatory Visits to Medicare Part B Beneficiaries

In [ ]:
# Identify column names 

list(data_all.columns) 

In [68]:
features = data_all[['Days_PM2.5', 'HHI']]

In [67]:
# get X and y variables

y = y_data
# output of model
X = features

In [70]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,Population,PovertyRate
224,5930538,9.726015
144,6402387,17.208144
108,3583561,9.888153
178,6028076,15.134033
73,9897264,15.984438


In [71]:
# scale the data
X_scaler =  StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [72]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, encoded_y_train)
predictions = model.predict(X_test_scaled)
model

SVC(kernel='linear')

In [74]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.0
Testing Data Score: 0.0


In [76]:
# Calculate classification report - determines whether it is a good fit for your model

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         8.0       0.00      0.00      0.00       0.0
        11.0       0.00      0.00      0.00       0.0
        12.0       0.00      0.00      0.00       0.0
        14.0       0.00      0.00      0.00       0.0
        18.0       0.00      0.00      0.00       0.0
        23.0       0.00      0.00      0.00       0.0
        24.0       0.00      0.00      0.00       0.0
        26.0       0.00      0.00      0.00       0.0
        29.0       0.00      0.00      0.00       0.0
        37.0       0.00      0.00      0.00       0.0
        41.0       0.00      0.00      0.00       0.0
        43.0       0.00      0.00      0.00       0.0
        44.0       0.00      0.00      0.00       0.0
        48.0       0.00      0.00      0.00       0.0
        58.0       0.00      0.00      0.00       0.0
        59.0       0.00      0.00      0.00       0.0
        65.0       0.00      0.00      0.00       0.0
        66.0       0.00    

C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Trial 3: Comparing Pollutants (Nitrogen Dioxide and Sulfur Dioxide) Levels to Household Income

In [78]:
# Identify column names 

list(data_all.columns) 

['Year',
 'State',
 'Population',
 'HHI',
 'PovertyRate',
 'beneficiaries_part_b',
 'one_ambulatory_visit',
 'diabetic_enrollees_age_65_to_75',
 'average_diabetic_enrollees_hemoglobin_a1c_test',
 'average_diabetic_enrollees_eye_exam',
 'average_diabetic_enrollees_blood_lipids_test',
 'average_female_enrollees_age_67_to_69',
 'average_female_age_67_to_69_mammogram',
 'beneficiaries_part_a_eligible',
 'leg_amputations_per_1000_enrollees',
 'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees',
 'State',
 'Days_with_AQI',
 'Year',
 'Good_Days',
 'Moderate_Days',
 'Unhealthy_Days',
 'Very_Unhealthy_Days',
 'Hazardous_Days',
 'Days_CO',
 'Days_NO2',
 'Days_Ozone',
 'Days_SO2',
 'Days_PM2.5',
 'Days_PM10']

In [79]:
features = data_all[['Days_NO2', 'Days_SO2']]

In [80]:
# get X and y variables

y = data_all["HHI"]
# output of model
X = features

In [82]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,Days_NO2,Days_SO2
224,16.25,0.12
144,4.28,5.52
108,5.75,0.12
178,3.41,22.14
73,1.36,12.50


In [84]:
# scale the data
X_scaler =  StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [85]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, encoded_y_train)
predictions = model.predict(X_test_scaled)
model

SVC(kernel='linear')

In [87]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

Training Data Score: 0.0
Testing Data Score: 0.0


In [89]:
# Calculate classification report - determines whether it is a good fit for your model

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           2       0.00      0.00      0.00       0.0
           3       0.00      0.00      0.00       0.0
           4       0.00      0.00      0.00       0.0
           5       0.00      0.00      0.00       0.0
           6       0.00      0.00      0.00       0.0
          15       0.00      0.00      0.00       0.0
          19       0.00      0.00      0.00       0.0
          20       0.00      0.00      0.00       0.0
          23       0.00      0.00      0.00       0.0
          24       0.00      0.00      0.00       0.0
          25       0.00      0.00      0.00       0.0
          26       0.00      0.00      0.00       0.0
          28       0.00      0.00      0.00       0.0
          29       0.00      0.00      0.00       0.0
          34       0.00      0.00      0.00       0.0
          39       0.00      0.00      0.00       0.0
          40       0.00    

C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [90]:
beneficiaries = data_all["HHI"]

In [91]:
target = data_all["HHI"]
HHI_list = data_all["HHI"].tolist()
target_names = [HHI_list]

In [92]:
hhi_data = data_all.drop("HHI", axis=1)
# data = data.drop("state_name", axis=1)
feature_names = data.columns
hhi_data

,Year,State,Population,PovertyRate,beneficiaries_part_b,one_ambulatory_visit,diabetic_enrollees_age_65_to_75,average_diabetic_enrollees_hemoglobin_a1c_test,average_diabetic_enrollees_eye_exam,average_diabetic_enrollees_blood_lipids_test,...,Moderate_Days,Unhealthy_Days,Very_Unhealthy_Days,Hazardous_Days,Days_CO,Days_NO2,Days_Ozone,Days_SO2,Days_PM2.5,Days_PM10
0,2011,Alabama,4747424,17.133186,478784.0,82.5,69691.0,83.9,63.3,80.3,...,72.37,0.37,0.00,0.37,0.11,0.00,158.63,0.16,84.89,4.74
1,2011,Alaska,700703,9.292239,49648.0,70.9,5449.0,74.5,55.8,66.9,...,34.88,0.62,0.12,0.62,2.12,0.00,75.62,0.00,176.25,20.50
2,2011,Arizona,6337373,15.835820,465298.0,78.2,50991.0,78.5,66.1,75.8,...,117.00,2.38,1.00,2.38,0.00,3.15,167.15,16.54,27.77,106.00
3,2011,Arkansas,2895928,17.846507,327939.0,80.7,40202.0,82.5,66.1,76.2,...,59.18,0.12,0.00,0.12,0.00,3.00,108.76,14.76,90.35,0.00
4,2011,California,36969200,14.096818,2238140.0,72.7,243999.0,80.6,64.1,78.0,...,120.37,7.02,0.37,7.02,0.17,6.91,197.43,0.09,129.24,11.81
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,2015,Virginia,8256630,11.164628,832699.0,83.3,98165.0,87.6,70.0,82.0,...,32.34,0.00,0.00,0.00,1.14,35.06,141.26,0.97,73.66,8.91
251,2015,Washington,6985464,13.005750,610922.0,78.5,56474.0,86.6,67.7,76.3,...,48.14,1.03,0.07,1.03,0.00,2.69,47.45,0.00,290.21,5.48
252,2015,West Virginia,1851420,17.466809,206961.0,80.8,29239.0,84.7,59.9,78.2,...,45.20,0.07,0.00,0.07,0.13,0.00,145.53,40.47,81.20,13.13
253,2015,Wisconsin,5742117,12.614651,474364.0,79.8,46596.0,91.0,69.7,81.6,...,42.68,0.07,0.00,0.07,0.00,0.71,191.43,14.54,84.86,0.25


In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [94]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.0

In [95]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.0

In [96]:
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.04791270493710223, 'Days_PM2.5'),
 (0.04668480724985405, 'one_ambulatory_visit'),
 (0.0462556784761781, 'PovertyRate'),
 (0.044698863180998714, 'HHI'),
 (0.044695344923325296, 'average_diabetic_enrollees_eye_exam'),
 (0.04367918582235266, 'average_diabetic_enrollees_blood_lipids_test'),
 (0.04292719859937989,
  'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees'),
 (0.04285983763303165, 'average_diabetic_enrollees_hemoglobin_a1c_test'),
 (0.042750697718652655, 'Good_Days'),
 (0.04129247766660115, 'average_female_age_67_to_69_mammogram'),
 (0.04123145036448092, 'average_female_enrollees_age_67_to_69'),
 (0.04039483207779299, 'Days_Ozone'),
 (0.04022001670010389, 'Population'),
 (0.04016585794085584, 'Days_with_AQI'),
 (0.03907527826773087, 'diabetic_enrollees_age_65_to_75'),
 (0.03853306165698141, 'beneficiaries_part_a_eligible'),
 (0.037707573303088465, 'Moderate_Days'),
 (0.03760785639468704, 'leg_amputations_per_1000_enrollees'),
 (0.03490203963646301, 'Days_NO2'

### Trial 4: Comparing Income and Pollutants (Nitrogen Dioxide) to Poverty Rate

In [97]:
# Identify column names 

list(data_all.columns) 

['Year',
 'State',
 'Population',
 'HHI',
 'PovertyRate',
 'beneficiaries_part_b',
 'one_ambulatory_visit',
 'diabetic_enrollees_age_65_to_75',
 'average_diabetic_enrollees_hemoglobin_a1c_test',
 'average_diabetic_enrollees_eye_exam',
 'average_diabetic_enrollees_blood_lipids_test',
 'average_female_enrollees_age_67_to_69',
 'average_female_age_67_to_69_mammogram',
 'beneficiaries_part_a_eligible',
 'leg_amputations_per_1000_enrollees',
 'discharges_ambulatory_care_sensitive_conditions_per_1000_enrollees',
 'State',
 'Days_with_AQI',
 'Year',
 'Good_Days',
 'Moderate_Days',
 'Unhealthy_Days',
 'Very_Unhealthy_Days',
 'Hazardous_Days',
 'Days_CO',
 'Days_NO2',
 'Days_Ozone',
 'Days_SO2',
 'Days_PM2.5',
 'Days_PM10']

In [98]:
features = data_all[['HHI', 'Days_NO2']]

In [99]:
# get X and y variables

y = data_all["PovertyRate"]
# output of model
X = features

In [100]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,HHI,Days_NO2
224,74551,16.25
144,44298,4.28
108,69461,5.75
178,47764,3.41
73,48471,1.36


In [101]:
# scale the data
X_scaler =  StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [102]:
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train_scaled, encoded_y_train)
predictions = model.predict(X_test_scaled)
model

SVC(kernel='linear')

In [104]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test)}")

ValueError: Classification metrics can't handle a mix of continuous and multiclass targets

In [106]:
# Calculate classification report - determines whether it is a good fit for your model

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

ValueError: Classification metrics can't handle a mix of continuous and multiclass targets

In [107]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
cv = StratifiedShuffleSplit(n_splits=2, test_size=175, random_state=42)
grid = GridSearchCV(model, param_grid, verbose=3, cv=cv)

In [108]:
# Train the model with GridSearch

grid.fit(X_train_scaled, encoded_y_train)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [109]:
# List the best parameters for this dataset
print(grid.best_params_)

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [110]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [111]:
print(predictions)

[ 76 183 108 180  89 130   6  57  58 182  97  19  66  55 108   9 109  77
 120   1 185  50  26 119  39  61   6 132  73 171  34 154 160 136  32  55
  58  22   1  58  26 141 121  95  93  86  71  74  78 146   1  89  78  39
 151 106  76  77  55  46 110  71 126   1]


In [114]:
# Calculate classification report - determines whether it is a good fit for your model

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

ValueError: Classification metrics can't handle a mix of continuous and multiclass targets

### Logistical Regression (does not work)

In [116]:
# Assign X (data) and y (target)
X = data_all.drop("State", axis=1)
y = data_all["HHI"]
print(X.shape, y.shape)

(255, 28) (255,)


In [117]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [118]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [119]:
classifier.fit(X_train, y_train)

C:\Users\me\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

### Kmeans Clusters (does not work)

In [120]:
# Create a kmeans model using k = 12
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=12)

# Fit the model to the data
kmeans.fit(X)

# Use the data to predict the clusters
# save the predictions as `predicted_clusters`
predicted_clusters = kmeans.predict(X)

In [121]:
# Print the cluster centers and cluster labels
centers = kmeans.cluster_centers_
labels = kmeans.labels_

In [122]:
centers

array([[2.01300000e+03, 7.78962600e+05, 5.71809111e+04, 1.25645155e+01,
        8.42137556e+04, 7.74488889e+01, 8.90148889e+03, 8.27644444e+01,
        6.90933333e+01, 7.37733333e+01, 7.49897778e+03, 6.44777778e+01,
        8.94799111e+04, 5.10444444e-01, 4.89888889e+01, 3.28428444e+02,
        2.01300000e+03, 2.54675556e+02, 6.86946667e+01, 1.04955556e+00,
        4.00000000e-02, 1.04955556e+00, 2.56222222e-01, 6.47377778e+00,
        1.72396000e+02, 4.98977778e+00, 1.25283111e+02, 1.90284444e+01],
       [2.01300000e+03, 1.93127789e+07, 5.27404000e+04, 1.52113288e+01,
        1.65199170e+06, 7.63000000e+01, 1.85730900e+05, 8.55500000e+01,
        6.95600000e+01, 8.46500000e+01, 1.34007200e+05, 6.51100000e+01,
        1.71509350e+06, 5.69000000e-01, 5.58500000e+01, 3.54244000e+02,
        2.01300000e+03, 3.00358000e+02, 5.15130000e+01, 1.56000000e-01,
        6.00000000e-03, 1.56000000e-01, 1.66000000e-01, 4.63700000e+00,
        2.37882000e+02, 7.48400000e+00, 9.94860000e+01, 4.58800

In [123]:
plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='viridis')
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)

plt.show()

TypeError: '(slice(None, None, None), 0)' is an invalid key

### Tune Model Parameters using GridSearch (not working)

#### Grid selection test

In [ ]:
target = data_all["beneficiaries_part_b"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [ ]:
# Support vector machine linear classifier

from sklearn.svm import SVC 
model = SVC(kernel='linear')

In [ ]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'gamma': [0.0001, 0.001, 0.01]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
# List the best parameters for this dataset

print(grid.best_params_)

In [ ]:
# Make predictions with the hypertuned model

predictions = grid.predict(X_test)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

In [ ]:
# Calculate classification report

from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

#### Grid selection test2

In [ ]:
# Split data into training and testing

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create the SVC Model. Can modify parameters to get the best score

from sklearn.svm import SVC 
model = SVC(kernel='linear')
model

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters

grid.fit(X_train, y_train)

#### Grid selection test3

In [ ]:
# Create the GridSearchCV model

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch

grid.fit(X_train_scaled, encoded_y_train)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

# use different model or change the features around to get a better score
# start off small .90

In [ ]:
# save the model

import joblib
filename = 'svm_model.sav'
joblib.dump(model, filename)